In [1]:
!pip install -q tiktoken openai langchain langchain_community langchain-openai faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.2 MB/s eta 0:00:00


In [2]:
import os
import re
import openai
from google.colab import userdata
from IPython.display import Markdown

In [3]:
#@title Инициируем ключи OpenAI через секрет
key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = key

In [23]:
# Фнкция приведения отвеа к единому стилю
def answer_style(system_prompt, answer, ass="",
                        model='gpt-4o-mini',
                        temp=0):
  #  docs = db.similarity_search("", k=5)
   # docs_competitor= db_competitor.similarity_search("", k=5)
    #all_texts = "\n\n".join([doc.page_content for doc in docs])
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": answer},
        {"role": "assistant", "content": ass}
     ]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temp
    )
    return response.choices[0].message.content

In [52]:
questions = {
   'Можно ли с педагогическим образованием переучиться на тракториста?': '''
                                                                          Да, можно сменить профессию с педагогической деятельности на работу трактористом. Для этого потребуется пройти специальное обучение и получить удостоверение тракториста-машиниста соответствующей категории.
                                                                        ''' ,
   'Что дает профессиональна переподготовка?': '''
                                               Профессиональная переподготовка за короткий срок позволяет освоить новую профессию на базе уже имеющегося образования.
                                               Программы профпереподготовки учитывают требования к профессии и направлены на формирование конкретных компетенций для практической работы.
                                               По окончании программ профессиональной переподготовки слушатель получает диплом установленного образца.
                                              ''',
  'Чем отличается профессиональная переподготовка от повыщения квалификаци?': '''
                                                                              Профессиональная переподготовка – это вид дополнительного профессионального образования, который позволяет специалисту сменить свою профессию или специальность.
                                                                              Другими словами, переподготовка – это сокращенная версия второго высшего или второго среднего профессионального образования.
                                                                              Вместо того, чтобы получать вторую специальность в вузе, можно пройти переподготовку, что дешевле и быстрее.
                                                                              Курсы профессиональной переподготовки включают программы с нагрузкой от 250 академических часов и более. По окончании выдается диплом о профессиональной переподготовке, дающий право заниматься новой профессиональной деятельностью.
                                                                              Повышение квалификации – это способ усовершенствования знаний и навыков специалистов.
                                                                              Технологии меняются, ежегодно в отраслях вводятся новые требования и стандарты. Чтобы быть в курсе этих изменений, необходимо проходить повышение квалификации.
                                                                              По профстандартам и квалификационным характеристикам некоторые специалисты обязаны проходить такие курсы с определённой периодичностью.
                                                                              Программы повышения квалификации включают учебные программы с нагрузкой от 16 часов. По завершении обучения выдается удостоверение о повышении квалификации.
                                                                              ''',
  'Чем отличается диплом о профессиональной переподготовки от второго высшего образования?': '''
                                                                                             Федеральный закон от 29.12.2012 г. № 273-ФЗ «Об образовании в Российской Федерации» дает определения:
                                                                                            второе высшее образование направлено на подготовку высококвалифицированных кадров (часть 1 статьи 69 Закона от 29.12.2012 г. № 273-ФЗ).
                                                                                            профессиональная переподготовка направлена на получение компетенции, необходимой для выполнения нового вида профессиональной деятельности, приобретение новых навыков, новой квалификации (часть 5 статьи 76 Закона от 29.12.2012 г. № 273-ФЗ).
                                                                                            Второе высшее образование Вы можете получить только в образовательной организации высшего образования (пункт 4 часть 2 статья 23 Закона от 29.12.2012 г. № 273-ФЗ), профессиональную переподготовку можно получить в любых других образовательных организациях (части 6, 10 статьи 76 Закона от 29.12.2012 г. № 273-ФЗ).
                                                                                             ''',
 'Можно ли получить удостоверение о повышении квалификации или диплом о профессиональной переподготовке без прохождения итоговой аттестации?': '''
    `                                                                                                                                          Нет. Освоение дополнительных профессиональных образовательных программ всегда завершается итоговой аттестацией обучающихся.
                                                                                                                                               Образовательная организация самостоятельно определяет форму итоговой аттестации (зачет, экзамен, защита выпускной работы и т.п.).
                                                                                                                                               ''',

 'Кто может пройти профессиональную переподготовку?': '''
                                                     Курсы профессиональной переподготовки рассчитаны на специалистов, имеющих высшее или среднее специальное образование, а также студентов старших курсов учебных заведений, желающих сменить профессию или сферу деятельности.
                                                     Профессиональная переподготовка необходима тем, кто имеет высшее образование, но по какой-то причине не может работать по специальности или стремится увеличить свои профессиональные ресурсы.
                                                     Обучение на курсах профессиональной переподготовки предоставляет возможность получить все необходимые теоретические и практические знания, требуемые для качественного выполнения рабочих обязанностей, за минимальный срок.
                                                     ''',

'Что такое и как проходит дистанционное обучение?': '''
                                                    Дистанционное обучение – это особый способ получения образования, во время которого применяются различные информационные и коммуникационные технологические разработки.
                                                    Удаленное обучение позволяет: проходить образовательные программы без необходимости посещать университеты и институты; получить круглосуточный доступ к порталу с учебно-методическими материалами в электронном виде; обучаться согласно личному графику в удобное время и сдавать тестирования удаленно.
                                                    ''',

'Какие документы необходимо запросить у клиента?': '''
                                                   Чтобы заключить договор на обучение, заявитель должен предоставить скан-копии некоторых документов:\n\nУдостоверение личности.\nДокумент о высшем или средне-специальном образовании.
                                                   Для специалистов в области медицины, которые планируют повысить квалификацию посредством курса, пакет необходимых для зачисления бумаг дополняется следующим:
                                                   Скан-копия действующего медицинского сертификата.\nЗаверенная копия трудовой книжки.\nСкан-копии документов о прохождении интернатуры и ординатуры.
                                                   ''',

'Что такое непрерывное медицинское обучение?': '''
                                               Непрерывное медицинское обучение подразумевает усовершенствованную форму постдипломного образования, которая дает возможность сотрудникам здравоохранения и фармацевтам непрерывно развиваться в своей работе, получая новые профессиональные знания и навыки. Такой подход позволяет сделать образовательную систему более индивидуальной, и получать знания, исходя из конкретных потребностей в той или иной области работы.
                                               Упраздненная сертификация специалистов медицинской и фармацевтической сфер была заменена допуском к процедуре аккредитации, для получения которого необходимо набрать определенное число баллов.
                                               ''',

'Кто входит в состав комиссии по проведению СОУТ от предприятия?': '''
                                                                    Как правило, приказом первого руководителя о создании комиссии в ее состав включают:
                                                                    инженера по охране труда;
                                                                    представителя профсоюзного органа;
                                                                    начальника структурного подразделения, где планируется проведение освидетельствования рабочих мест.
                                                                    ''',


'Что такое дополнительное профессиональное образование?': '''
                                                          Дополнительное профессиональное образование представляет собой обучение, направленное на углубление и расширение ранее полученных знаний и навыков специалистов различных профессий.
                                                          Его цель — повышение квалификации и профессиональная переподготовка сотрудников для решения новых задач и адаптации к изменениям рынка труда.
                                                          ''',
'Какие формы дополнительного профессионального образования существуют?': 'Повышение квалификации (краткосрочные курсы) Профессиональная переподготовка (получение новой специальности).',
'Кто может пройти дополнительное профессиональное образование?': '''
                                                                  Любое лицо, имеющее среднее специальное или высшее образование, вправе пройти дополнительную подготовку или повысить квалификацию по интересующей профессии.
                                                                  Обычно наличие опыта работы не обязательно, однако некоторые направления требуют определённого стажа.
                                                                  ''',

'Какой объем часов необходим для получения удостоверения или диплома?': '''
                                                                        Минимальное количество учебных часов устанавливается государством: Программа повышения квалификации должна составлять минимум 16 академических часов.
                                                                        Курсы профессиональной переподготовки предполагают не менее 250 часов обучения.
                                                                        '''  ,


 'Признаются ли полученные дипломы и сертификаты?': '''
                                                    Документы государственного образца признаются всеми работодателями на территории Российской Федерации.
                                                    Частные институты могут выдавать собственные сертификаты, которые тоже имеют значение, особенно если учреждение аккредитовано соответствующими ведомствами.
                                                    ''',
'Какая ответственность, если сотрудники организации не прошли обучение по пожарной безопасности?': '''
                                                                                                  Если работники организации не прошли обучение по пожарной безопасности, но продолжают трудовую деятельность в организации, руководитель организации привлекается к ответственности.
                                                                                                  Статья 20.4 (части 1 и 2) КоАП РФ предусматривает за подобное нарушение штраф в размере до 200 тыс. рублей.
                                                                                                  ''',

'Почему важна специальная оценка условий труда?': '''
                                                    Специальная оценка условий труда необходима для выявления вредных факторов производства, оценки рисков здоровью работающих, разработки мер профилактики профзаболеваний и травм, правильного подбора индивидуальных средств защиты, обоснования льгот и компенсаций работникам, занятым на тяжелых работах.
                                                    ''',

'Кого привлекают к административной ответственности за нарушение норм охраны труда?': '''
                                                                                         Административной ответственности подлежат должностные лица предприятий и индивидуальные предприниматели. Для юридических лиц применяются штрафы и приостановка деятельности предприятия.
                                                                                        ''',

 'Какой документ подтверждает прохождение проверки знаний по охране труда?': 'Документ, подтверждающий проверку знаний работника по охране труда, называется протоколом заседания комиссии по проверке знаний требований охраны труда.',

 'Когда проводятся внеплановые инструктажи по охране труда?': '''
                                                             `Внеплановые инструктажи проводятся при изменении технологического процесса, замене оборудования, выявлении нарушений техники безопасности, поступлении новых сведений о травмоопасности на предприятии, изменении нормативных документов по охране труда, а также после аварий и несчастных случаев.
                                                             ''',
'Чем отличается первичный инструктаж от повторного?': '''
                                                      Первичный инструктаж проводят перед началом самостоятельной работы сотрудника непосредственно на рабочем месте руководителем подразделения или непосредственным начальником.
                                                      Повторный инструктаж проходит периодически раз в полгода-год с целью освежения и закрепления ранее полученных знаний по технике безопасности.
                                                      ''',

'Кто проводит вводный инструктаж по охране труда?': '''
                                                     Вводный инструктаж проводится специалистом службы охраны труда организации либо лицом, ответственным за охрану труда, назначенным приказом руководителя предприятия.
                                                     ''',

'Каковы обязанности работодателя в области охраны труда': '''
                                                          Обязанности работодателя включают обеспечение безопасности рабочих мест, организацию обучения сотрудников правилам охраны труда, проведение инструктажей и проверок знаний требований охраны труда, создание условий для своевременного оказания первой помощи пострадавшим, предоставление специальной одежды и СИЗ, контроль соблюдения работниками правил охраны труда.
                                                          ''',

'Какие права имеет работник в области охраны труда?': '''
                                                      Работник имеет право на рабочее место, соответствующее требованиям охраны труда, получение достоверной информации об условиях и охране труда на рабочем месте, обучение безопасным методам и приемам выполнения работ, средства индивидуальной защиты, медицинскую помощь и страхование от несчастных случаев на производстве и профессиональных заболеваний.
                                                      ''',

'Что такое охрана труда?': '''
                           Охрана труда — система сохранения жизни и здоровья работников в процессе трудовой деятельности, включающая правовые, социально-экономические, организационно-технические, санитарно-гигиенические, лечебно-профилактические, реабилитационные и другие мероприятия.
                           ''',

'Как получить налоговый вычет за ДПО?': '''
                                        1. Соберите необходимые документы: Документы, подтверждающие оплату (чеки, квитанции, анковские выписки). Договор с образовательным учреждением, подтверждающий факт прохождения курса дополнительного профессионального образования.
                                        Справка 2-НДФЛ от работодателя за тот период, когда были понесены расходы на обучение. Копия лицензии образовательного учреждения (необязательно, если лицензия указана в договоре).\n2. Заполните налоговую декларацию: Используйте программу ФНС «Налогоплательщик ЮЛ», сайт госуслуг или онлайн-сервис налоговой службы для заполнения декларации формы №3-НДФЛ.\n3. Подайте заявление на возврат налога: Заявление подается вместе с декларацией и комплектом документов в территориальное отделение Федеральной налоговой службы либо через личный кабинет налогоплательщика на сайте nalog.ru.
                                        Ожидайте рассмотрения заявления: Срок проверки представленных документов составляет три месяца, после чего принимается решение о предоставлении налогового вычета.
                                         '''
}

In [54]:
assist = "Что такое охрана труда? Охрана труда — система сохранения жизни и здоровья работников в процессе трудовой деятельности, включающая правовые, социально-экономические, организационно-технические, санитарно-гигиенические, лечебно-профилактические, реабилитационные и другие мероприятия."
system = "Ты — эксперт по стандартизации клиентской коммуникации: совмещаешь роли корректора, стилиста, методиста и редактора"

In [27]:
answer_style(system, user, assist)

'Профессиональная переподготовка позволяет быстро освоить новую профессию на основе уже имеющегося образования. Программы направлены на формирование конкретных навыков для практической работы. По окончании обучения вы получаете диплом установленного образца. \n\nПовышение квалификации, в свою очередь, помогает углубить знания и навыки в уже имеющейся профессии, но не меняет её.'

In [55]:
for i, (key, value) in enumerate(questions.items(), start=1):
  user  = f"Твоя задача - скорректироваать ответ консультанта клитенту,в соответствии с предложиным стилем ответа. Ответ должен быть краток и понятен человеку. Вопрос: {key} ответ {value}"
  a = answer_style(system, user, assist)
  display(Markdown(f"Ответ на вопрос, {i}"))
  display(Markdown(a))



Ответ на вопрос, 1

Да, с педагогическим образованием можно переучиться на тракториста. Для этого нужно пройти специальное обучение и получить удостоверение тракториста-машиниста соответствующей категории.

Ответ на вопрос, 2

Профессиональная переподготовка позволяет быстро освоить новую профессию, основываясь на уже имеющемся образовании. Программы учитывают требования к профессии и формируют необходимые компетенции для практической работы. По окончании обучения вы получите диплом установленного образца.

Ответ на вопрос, 3

...

Работник имеет следующие права в области охраны труда:

1. Рабочее место, соответствующее требованиям охраны труда.
2. Достоверная информация об условиях труда.
3. Обучение безопасным методам работы.
4. Средства индивидуальной защиты.
5. Медицинская помощь.
6. Страхование от несчастных случаев и профессиональных заболеваний.

Ответ на вопрос, 25

Охрана труда — это система, направленная на сохранение жизни и здоровья работников в процессе их трудовой деятельности. Она включает в себя правовые, организационные, санитарные и профилактические меры.

Ответ на вопрос, 26

Чтобы получить налоговый вычет за ДПО, выполните следующие шаги:

1. **Соберите документы**:
   - Чеки и квитанции об оплате.
   - Договор с образовательным учреждением.
   - Справка 2-НДФЛ от работодателя за период обучения.
   - Копия лицензии образовательного учреждения (если не указана в договоре, то не обязательно).

2. **Заполните налоговую декларацию**:
   - Используйте программу ФНС «Налогоплательщик ЮЛ», сайт госуслуг или онлайн-сервис налоговой службы для заполнения формы 3-НДФЛ.

3. **Подайте заявление на возврат налога**:
   - Заявление подается вместе с декларацией и документами в территориальное отделение ФНС или через личный кабинет на сайте nalog.ru.

4. **Ожидайте рассмотрения**:
   - Срок проверки документов — три месяца. После этого будет принято решение о предоставлении вычета.

Ссылка на таблицу с фопросами в единой форме https://docs.google.com/spreadsheets/d/1rG0c4HJPQYudUVeNd1MMGcGUou6vjSreBlHy4JqS6ck/edit?usp=sharing